# 자료구조 6주차 - 1: Deque,

## 2020년 10월 06일 안상호

> 덱과 우선순위큐
 
1. 덱 소개 & 구현
2. 우선순위큐 소개 & 구현
3. 응용 문제
4. 코테 문제


---




# 1. 덱소개 & 구현



## 1.1. 덱소개

- 스택이나 큐보다 입출력이 자유로운 자료구조
- 덱(deque)은 double-ended queue의 줄임말
    + 전단과 후단에서 모두 삽입과 삭제가 가능한 큐
    + 여전히 중간에 넣거나 빼지는 못함
    
    
    
### 추상 자료형

- 데이터: 전단과 후단을 통한 접근을 허용하는 항목들의 모음

- 연산
    + `Deque()`: 비어 있는 새로운 덱을 만든다.
    + `isEmpty()`: 덱이 비어있으면 True 아니면 False
    + `addFront(x)`: 항목 x를 덱의 맨 앞에 추가한다.
    + `deleteFront()`: 큐의 맨 앞에 있는 항목을 꺼내 반환한다.
    + `getFront()`: 큐의 맨 앞에 있는 항목을 삭제하지 않고 반환한다.
    + `addRear(x)`: 항목 x를 덱의 맨 뒤에 추가한다.
    + `deleteRear()`: 맨 뒤의 항목을 꺼내지 않고 반환한다.
    + `getRear()`: 맨 뒤의 항목을 꺼내지 않고 반환한다.
    + `isFull()`: 덱이 가득 차 있으면 True를 아니면 False를 반환한다. 
    + `size()`: 덱의 모든 항목들의 개수를 반환
    + `clear()`: 덱을 공백상태로 만든다.
    
### 원형 덱의 연산

- 큐와 데이터는 동일함
- 연산은 유사함
- 큐와 알고리즘이 동일한 연산
    + addRear(), deleteFront(), getFront(): 큐의 enqueue, dequeue, peek 연산과 동일    
    + 덱의 후단을 스택의 상단으로 사용하면 addRear(), deleteRear(), getRear() 연산은 스택의 push, pop, peek 연산과 정확히 동일하다.

### 원형 큐에서 추가된 연산
- delete_rear(), add_front(), get_rear()
    + 반 시계방향 회전 필요
    
    

## 1.2. 덱의 구현 (상속)

원형 큐를 상속하여 원형 덱 클래스를 구현

In [1]:
MAX_QSIZE = 10

class CircularQueue:
    def __init__(self, MAX_QSIZE):
        self.front = 0
        self.rear = 0
        self.MAX_QSIZE = MAX_QSIZE
        self.items = [None] * self.MAX_QSIZE
        
    def isEmpty(self): return self.front == self.rear
    def isFull(self): return self.front == (self.rear+1) % self.MAX_QSIZE
    def clear(self): self.front = self.rear
        
        
    def enqueue(self, item):
        """
        포화상태가 아니면 rear를 회전하고,
        rear 위치에 item 삽입
        """
        if not self.isFull():
            self.rear = (self.rear+1) % self.MAX_QSIZE
            self.items[self.rear] = item 
    
    def dequeue(self):
        """
        공백상태가 아니면 front를 회전하고,
        front위치의 항목을 반환
        """
        if not self.isEmpty():
            self.front = (self.front+1) % self.MAX_QSIZE
            return self.items[self.front]
        
    def peek(self):
        if not self.isEmpty():
            return self.items[(self.front+1) % self.MAX_QSIZE ]
    
    def size(self):
        return (self.rear - self.front + self.MAX_QSIZE) % self.MAX_QSIZE
    
    def display(self):
        out = []
        if self.front < self.rear:
            out = self.items[self.front+1:self.rear+1]
        else:
            out = self.items[self.front+1:self.MAX_QSIZE] + self.items[0:self.rear+1]
            
        print(f"[f={self.front}, r={self.rear}] ==> {out}")

In [2]:
class CircularDeque(CircularQueue):               # CircularQueue에서 상속
    def __init__(self, MAX_QSIZE):                # CircularDeque의 생성자 
        super().__init__(MAX_QSIZE)               # 생성자는 상속이 안되니까 재정의, 부모 클래스의 생성자를 호출함
    
    # 인터페이스 변경 멤버들
    def addRear(self, item): self.enqueue(item)   # enqueue 호출 
    def deleteFront(self): return self.dequeue() # 반환에 주의
    def getFront(self): return self.peek()       # 반환에 주의
    
    # 추가로 구현할 메소드
    def addFront(self, item):                     # 새로운 기능: 전단 삽입
        if not self.isFull():
            self.items[self.front] = item         # 항목 저장
            self.front = self.front - 1           # 반시계 방향으로 회전
            if self.front < 0: self.front = self.MAX_QSIZE - 1
                
    def deleteRear(self):                         # 새로운 기능: 후단 삭제
        if not self.isEmpty():     
            item = self.items[self.rear]          # 항목 복사
            self.rear = self.rear - 1             # 반시계 방향으로 회전
            if self.rear < 0: self.rear = self.MAX_QSIZE - 1
            return item                           # 항목 반환
        
    def getRear(self):                            # 새로운 기능: 후단 peek
        return self.items[self.rear]
            

In [5]:
dq = CircularDeque(MAX_QSIZE)       # 덱 객체 생성. f=r=0
for i in range(9):                 # i: 0, 1, 2, ..., 8
    if i%2 == 0: dq.addRear(i)     # 짝수는 후단에 삽입:
    else: dq.addFront(i)           # 홀수는 전단에 삽입:
dq.display()                        # front=6, rear=5

for i in range(2): dq.deleteFront() # 전단에서 두 번의 삭제: f=8
for i in range(3): dq.deleteRear()  # 후단에서 세 번의 삭제: r=2
dq.display()

for i in range(9,14): dq.addFront(i) # i: 9, 10, ..., 13: f=3
dq.display()

[f=6, r=5] ==> [7, 5, 3, 1, 0, 2, 4, 6, 8]
[f=8, r=2] ==> [3, 1, 0, 2]
[f=3, r=2] ==> [13, 12, 11, 10, 9, 3, 1, 0, 2]


# 2. 우선순위큐 소개 & 구현

- 실생활에서의 우선순위
    + 도로에서의 자동차 우선순위
- 우선순위 큐(priority queue)
    + 우선순위의 개념을 큐에 도입한 자료구조
    + 모든 데이터가 우선순위를 가짐
    + 입력 순서와 상관없이 우선순위가 높은 데이터가 먼저 출력
    + 가장 일반적인 큐로 볼 수 있다. Why?
- 응용분야
    + 시뮬레이션, 네트워크 트래픽 제어, OS의 작업 스케쥴링 등
    
    
### 우선순위 큐 ADT

- 데이터: 우선순위를 가진 요소들의 모음
- 연산
    + `PriorityQueue()`: 비어 있는 우선순위 큐를 만든다.
    + `isEmpty()`: 우선순위 큐가 공백상태인지를 검사한다.
    + `enqueue(e)`: 우선순위를 가진 항목 e를 추가한다.
    + `dequeue()`: 가장 우선순위가 높은 항목을 꺼내서 반환한다.
    
    + `peek()`: 가장 우선순위가 높은 요소를 삭제하지 않고 반환한다.
    + `size()`: 우선순위 큐의 모든 항목들의 개수를 반환한다.
    + `clear()`: 우선순위 큐를 공백상태로 만든다. 
    
    
- 구현방법: **배열 구조** / **연결된 구조** / **힙 트리**

In [1]:
# Python list를 이용한 Priority Queue ADT의 구현
class PriorityQueue:
    def __init__(self):
        self.items = []
        
    def isEmpty(self):
        return len(self.items) == 0
    
    def size(self): return len(self.items)  # 전체 항목의 개수
    def clear(self): self.items = []        # 초기화
        
    def enqueue(self, item):                 # 삽입 연산 
        self.items.append(item)              # 리스트의 맨 뒤에 삽입
        
    ## 삭제 연산
    def findMaxIndex(self):
        """
        최대 우선순위 항목의 인덱스 반환
        최고 우선순위 인덱스를 갱신하는 방식으로
        """
        if self.isEmpty(): return None
        else:
            highest = 0
            for i in range(1, self.size()):
                if self.items[i] > self.items[highest]:
                    highest = i
            return highest
            
    def dequeue(self):
        """
        삭제 연산
        """
        highest = self.findMaxIndex()        
        if highest is not None:
            return self.items.pop(highest)
        
    def peek(self):
        """
        peek 연산 
        """
        highest = self.findMaxIndex()         # 우선순위가 가장 높은 항목
        if highest is not None:
            return self.items[highest]      # 꺼내지 않고 반환

In [2]:
q = PriorityQueue()
for i in [34, 18, 27, 45, 15]:
    q.enqueue(i) 

print("PQueue: ", q.items)

while not q.isEmpty():
    print("Max Priority = ", q.dequeue())

PQueue:  [34, 18, 27, 45, 15]
Max Priority =  45
Max Priority =  34
Max Priority =  27
Max Priority =  18
Max Priority =  15


### 시간 복잡도

- 정렬되지 않은 리스트 사용
    + enqueue(): 대부분의 경우 $O(1)$
    + findMaxIndex(): $O(n)$
    + dequeue(), peek(): $O(n)$
- 정렬된 리스트 사용
    + enqueue(): $O(n)$
    + dequeue(), peek(): $O(1)$
    
- 힙 트리 (향후)
    + enqueue(), dequeue(): $O(logn)$
    + peek(): $O(1)$



---

# 3. 응용 문제

## 3.1. 우선순위 큐의 응용: 전략적인 미로 탐색

개념이 있는 쥐 

- 큐에 저장되는 항목: (x, y, -d) 형태의 튜플
    + 우선순위는 가장 높은 수이므로, 가장 적은 거리에 -를 붙여서 구현
    


In [19]:
import numpy as np

maze = np.array([['1', '1', '1', '1', '1', '1'],
                 ['e', '0', '0', '0', '0', '1'],
                 ['1', '0', '1', '0', '1', '1'],
                 ['1', '1', '1', '0', '0', 'x'],
                 ['1', '1', '1', '0', '1', '1'],
                 ['1', '1', '1', '1', '1', '1']])

MAZE_SIZE = maze.shape[1]
MAZE_SIZE

6

In [20]:
import math


class MazePQueue(PriorityQueue):
    """
    findMaxIndex만 override하여 
    미로 탐색을 위한 우선순위큐 클래스 정의
    """
    def __init__(self):
        super().__init__()
    
    def findMaxIndex(self):
        if self.isEmpty(): return None
        else:
            highest = 0
            for i in range(1, self.size()):
                if self.items[i][2] > self.items[highest][2]:
                    highest = i
            return highest


(ox, oy) = (5, 4)
def dist(x, y):
    (dx, dy) = (ox-x, oy-y)
    return math.sqrt(dx*dx + dy*dy)


#     def findMaxIndex(self):
#         if self.isEmpty(): return None
#         else:
#             highest = 0
#             for i in range(1, self.size()):
#                 if self.items[i][2] > self.items[highest][2]:
#                     highest = i
#             return highest


    

def isValidPos(x, y, maze):
    if x < 0 or y < 0 or x >= MAZE_SIZE or y >= MAZE_SIZE:
        return False
    else:
        return maze[y][x] == '0' or maze[y][x] == 'x'




def MySmartSearch(maze):
    q = MazePQueue()
    q.enqueue((0, 1, -dist(0, 1)))
    print("PQueue")
    
    while not q.isEmpty():
        here = q.dequeue()
        print(here[0:2], end='->')
        x,y,_ = here
        
        if (maze[y][x] == 'x'): return True, maze
        else:
            maze[y][x] = '.'
            if isValidPos(x, y - 1, maze): q.enqueue((x,y-1, -dist(x,y-1)))
            if isValidPos(x, y + 1, maze): q.enqueue((x,y+1, -dist(x,y+1)))
            if isValidPos(x - 1, y, maze): q.enqueue((x-1,y, -dist(x-1,y)))
            if isValidPos(x + 1, y, maze): q.enqueue((x+1,y, -dist(x+1,y)))
        print('우선순위큐: ', q.items)
    return False, maze

In [21]:


result, maze_map = MySmartSearch(maze.copy())

if result: print(' --> 미로탐색 성공')
else: print(' --> 미로탐색 실패')
    
print(maze_map)

PQueue
(0, 1)->우선순위큐:  [(1, 1, -5.0)]
(1, 1)->우선순위큐:  [(1, 2, -4.47213595499958), (2, 1, -4.242640687119285)]
(2, 1)->우선순위큐:  [(1, 2, -4.47213595499958), (3, 1, -3.605551275463989)]
(3, 1)->우선순위큐:  [(1, 2, -4.47213595499958), (3, 2, -2.8284271247461903), (4, 1, -3.1622776601683795)]
(3, 2)->우선순위큐:  [(1, 2, -4.47213595499958), (4, 1, -3.1622776601683795), (3, 3, -2.23606797749979)]
(3, 3)->우선순위큐:  [(1, 2, -4.47213595499958), (4, 1, -3.1622776601683795), (3, 4, -2.0), (4, 3, -1.4142135623730951)]
(4, 3)->우선순위큐:  [(1, 2, -4.47213595499958), (4, 1, -3.1622776601683795), (3, 4, -2.0), (5, 3, -1.0)]
(5, 3)-> --> 미로탐색 성공
[['1', '1', '1', '1', '1', '1'], ['.', '.', '.', '.', '0', '1'], ['1', '0', '1', '.', '1', '1'], ['1', '1', '1', '.', '.', 'x'], ['1', '1', '1', '0', '1', '1'], ['1', '1', '1', '1', '1', '1']]


### 우선순위 큐의 주요 응용

- (8.6 절)허프만 코딩 트리
- (11.3 절)Kruskal의 MST 알고리즘: 최소 비용 신장 트리
- (11.4 절)Dijkstra의 최단거리 알고리즘
- 인공지능의 A* 알고리즘